In [1]:
import numpy as np
import gamspy as gams
import pandas as pd
import sys

def print_records(vars):
    for k in np.arange(0,len(vars)):
        print(vars[k].name,':')
        display(vars[k].records)
        print()

m = gams.Container()

i = gams.Set(container=m,
             name='i',
             description='reactor'
            )
i.setRecords(['F1','F2','F3','S4'])  # F1: fermenter 1, F2: fermenter 2, 
                                     # F3: fermenter 3, S4: still 4

j = gams.Set(container=m,
             name='j',
             description='drink'
            )
j.setRecords(['A','B'])  # A: ale, B: bourbon

print_records([i,j])

cap = gams.Parameter(container=m,
                     name='cap',
                     domain=i,
                     description='weekly production capacity of reactor i in h/wk'
                    )

cap.setRecords([('F1',150),('F2',150),('F3',150),('S4',150)])

pft = gams.Parameter(container=m,
                     name='pft',
                     domain=j,
                     description='profit of drink j in $ per 100 gal'
                    )

pft.setRecords([('A',500),('B',1000)])

print_records([cap,pft])

hrs_matrix = pd.DataFrame(
    [
        ['F1','A',5],
        ['F1','B',10],
        ['F2','A',10],
        ['F2','B',5],
        ['F3','A',5],
        ['F3','B',15],
        ['S4','A',0],
        ['S4','B',20],
    ],
    columns=['reactor','drink','hours']
).set_index(['reactor','drink'])

hrs = gams.Parameter(container=m,
                     name='hrs',
                     domain=[i,j],
                     description='number of hours spent in each reactor by 100 gal of each drink',
                     records=hrs_matrix.reset_index()
                    )

print_records([hrs])

x = gams.Variable(container=m,
                  name='x',
                  domain=j,
                  type='Positive',
                  description='weekly production on drink j'
                 )

cap_const = gams.Equation(container=m,
                          name='cap_const',
                          domain=i,
                          description='observe capacity constraint of reactor i'
                         )

cap_const[i] = gams.Sum(j,hrs[i,j]*x[j]) <= cap[i]  # constraints

obj = gams.Sum(j,pft[j]*x[j])  # objective function

problem = gams.Model(container=m,  
                       name='distillery',
                       equations=[cap_const],
                       problem='LP',
                       sense=gams.Sense.MAX,
                       objective=obj
                    )  # problem

problem.solve(output=None, 
              options=gams.Options(equation_listing_limit=10, 
                                   variable_listing_limit=10
                                  )
             )  # use output=sys.stdout to see the raw GAMS output, useful for troubleshooting

print(problem.getEquationListing())
print()
# print(cap_const.getEquationListing())
# print(x.getVariableListing())
print_records([x])
print('Optimum: ',problem.objective_value)

i :


,uni,element_text
0,F1,
1,F2,
2,F3,
3,S4,



j :


,uni,element_text
0,A,
1,B,



cap :


...


hrs :


,reactor,drink,value
0,F1,A,5.0
1,F1,B,10.0
2,F2,A,10.0
3,F2,B,5.0
4,F3,A,5.0
5,F3,B,15.0
6,S4,A,0.0
7,S4,B,20.0



cap_const(F1)..  5*x(A) + 10*x(B) =L= 150 ; (LHS = 0)
cap_const(F2)..  10*x(A) + 5*x(B) =L= 150 ; (LHS = 0)
cap_const(F3)..  5*x(A) + 15*x(B) =L= 150 ; (LHS = 0)
cap_const(S4)..  20*x(B) =L= 150 ; (LHS = 0)
distillery_objective..  500*x(A) + 1000*x(B) - distillery_objective_variable =E= 0 ; (LHS = 0)

x :


,j,level,marginal,lower,upper,scale
0,A,12.0,0.0,0.0,inf,1.0
1,B,6.0,0.0,0.0,inf,1.0



Optimum:  12000.000000000002
